# 7. Binning Numeric Columns

Numeric columns like year might better be represented as a nominal categorical variable which are suitable for one-hot encoding. Encoding each unique year would yield a huge number of columns with some of them having very few non-zero entries. Instead, we can use a much smaller number of bins to contain groups of years. The `KBinsDescritizer` is a new transformer introduced in scikit-learn version 0.20 that does this.

In [1]:
import pandas as pd
hs = pd.read_csv('data/housing_sample.csv')
hs.head()

,Neighborhood,Exterior1st,YearBuilt,LotFrontage,GrLivArea,GarageArea,SalePrice
0,CollgCr,VinylSd,2003,65.0,1710,548,208500
1,Other,Other,1976,80.0,1262,460,181500
2,CollgCr,VinylSd,2001,68.0,1786,608,223500
3,Other,Other,1915,60.0,1717,642,140000
4,Other,VinylSd,2000,84.0,2198,836,250000


## Using `KBinsDescritizer`

The `KBinsDescritizer` transformer is found in the `preprocessing` module. To instantiate it you must choose the number of bins you'd like to divide your numeric data into and whether you want one-hot or ordinal encoding. The default value for `encode` is 'onehot' which returns a sparse array. We choose `onehot-dense` to return an actual numpy array. You can create the bins that have equally spaced edges, that have the same number of observations in each bin, or a more complex approach involving k-means.

In [2]:
X = hs[['YearBuilt']].values
y = hs.pop('SalePrice').values

Let's bin and encode the year built.

In [3]:
from sklearn.preprocessing import KBinsDiscretizer
kbd = KBinsDiscretizer(n_bins=6, encode='onehot-dense', strategy='uniform')
kbd.fit_transform(hs[['YearBuilt']])

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

You can see the edges of the bins with the `bin_edges_` attribute.

In [4]:
kbd.bin_edges_

array([array([1872., 1895., 1918., 1941., 1964., 1987., 2010.])],
      dtype=object)

## Putting it all together

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from mymetrics import root_mean_squared_log_error

# string pipeline
string_si = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe = OneHotEncoder(sparse=False)
steps = [('impute', string_si), ('encode', ohe)]
string_pipe = Pipeline(steps)

# numeric pipeline
numeric_si = SimpleImputer(strategy='mean')
ss = StandardScaler()
steps = [('si', numeric_si), ('standardize', ss)]
numeric_pipe = Pipeline(steps)

# year transformation
kbd = KBinsDiscretizer(n_bins=10, encode='onehot-dense', strategy='uniform')

# columns
string_cols = ['Neighborhood', 'Exterior1st']
numeric_cols = ['LotFrontage', 'GrLivArea', 'GarageArea']

transformers = [('string', string_pipe, string_cols), 
                ('numeric', numeric_pipe, numeric_cols), 
                ('year', kbd, ['YearBuilt'])]

ct = ColumnTransformer(transformers)
rfr = RandomForestRegressor()
steps = [('transformers', ct), ('rfr', rfr)]
final_pipe = Pipeline(steps)

kf = KFold(n_splits=5, shuffle=True)
grid = {'transformers__numeric__si__strategy': ['mean', 'median'],
       'rfr__n_estimators': [50, 100], 'rfr__max_depth': range(2, 6)}
gs = GridSearchCV(final_pipe, grid, cv=kf, scoring=root_mean_squared_log_error)
gs.fit(hs, y)
gs.best_params_

{'rfr__max_depth': 5,
 'rfr__n_estimators': 100,
 'transformers__numeric__si__strategy': 'median'}

## Exercise 

Use `KBinsDescritizer` experimenting with the number of bins and the binning strategy.